<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_07_3_search_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 7: LangChain: Agents**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 7 Material

* Part 7.1: Introduction to LangChain Agents [[Video]]() [[Notebook]](t81_559_class_07_1_agents.ipynb)
* Part 7.2: Understanding LangChain Agent Tools [[Video]]() [[Notebook]](t81_559_class_07_2_tools.ipynb)
* **Part 7.3: LangChain Retrival and Search Tools** [[Video]]() [[Notebook]](t81_559_class_07_3_search_tools.ipynb)
* Part 7.4: Constructing LangChain Agents [[Video]]() [[Notebook]](t81_559_class_07_4_more_agent.ipynb)
* Part 7.5: Custom Agents [[Video]]() [[Notebook]](t81_559_class_07_5_custom_agent.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai langchain_experimental duckduckgo-search langchainhub sentence-transformers chromadb

Note: using Google CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.6 MB/s eta 0:00:00
 

# 7.3: LangChain Retrival and Search Tools

In this section we will look at two forms of retrival for use with LangChain agents: search and retrival tools. Search tools allow your agent to access search engines such as Google to provide very current information to the agent. Additionally, you can use RAG to augment your agent's knowledge with a vectorized document store, similar to previous RAG examples in this course.

## Utilizing Duck Duck Go Search

DuckDuckGo is a search engine designed with a strong emphasis on user privacy. Unlike many mainstream search engines, DuckDuckGo does not track your searches or store personal information, ensuring a more secure and private online experience. Although you may not have heard of DuckDuckGo, it has gained popularity for its straightforward approach to search and its commitment to user privacy.

For many examples in this course, we will use [DuckDuckGo](https://duckduckgo.com/) for several important reasons. Firstly, its API is free to access and does not require an authentication key, simplifying the process and allowing us to dive straight into coding without the hassle of complex setups. Secondly, using DuckDuckGo will help us stay mindful of privacy issues and understand the importance of protecting personal data in our digital interactions. This practical experience will not only improve our technical skills but also broaden our knowledge of the diverse tools available in the digital ecosystem.

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
search_tool.run("Who is the president of the USA?")

'Joe Biden (born November 20, 1942, Scranton, Pennsylvania, U.S.) 46th president of the United States (2021- ) and 47th vice president of the United States (2009-17) in the Democratic administration of Pres. Barack Obama. He previously represented Delaware in the U.S. Senate (1973-2009). Basic Qualifications: Age, Citizenship and Residency. According to the Constitution, there are just three basic qualifications for being president: Age: The person must be at least 35 years old. Former President Donald Trump and President Joe Biden faced off on Thursday night during the first presidential debate of the 2024 race for the White House. The CNN Presidential Debate marks the ... The president of the United States is the: U.S. head of state; Chief executive of the federal government; Commander-in-Chief of the armed forces; Current president. The 46th and current president of the United States is Joseph R. Biden, Jr. He was sworn into office on January 20, 2021. Former U.S. presidents. The Un

In [3]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

search_tool = DuckDuckGoSearchRun()

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [search_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "Return the value of the DJIA as a floating point number, return just the number, no text or comments."})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'current value of DJIA'}`


Dow Jones Industrial Average + Add to watchlist + Add an alert. DJI:DJI. Dow Jones Industrial Average. Actions. Add to watchlist; Add an alert; Price (USD) 39,411.21; Today's Change 260.88 / 0.67%; Shares traded 387.00m; ... Share price information may be rounded up/down and therefore not entirely accurate. Dow Jones Today: Get all information on the Dow Jones Index including historical chart, news and constituents. The Dow Jones Industrial Average ( DJINDICES:^DJI) is a stock index that tracks 30 of the largest U.S. companies. Created in 1896, it is one of the oldest stock indexes, and its performance is ... Headlines for Dow Jones Industrial Average (Dow Jones Global:DJIA) News for Dow Jones Industrial Average. Thursday, June 27, 2024. 04:44 PM ET. Hedge funds are dumping tech stocks despite their recent rally, says Goldman Sachs MarketWatch. ... Historical and current e

{'input': 'Return the value of the DJIA as a floating point number, return just the number, no text or comments.',
 'output': '39411.21'}

## Using Tavily Search

We will now explore how to use [Tavily](https://tavily.com/) with LangChain. While this course will primarily focus on using DuckDuckGo due to its free access, it’s important to note that Tavily also offers a free version. The free plan of Tavily allows users to make up to 1,000 API calls per month without requiring a credit card, making it accessible for those who want to experience its advanced search capabilities tailored for large language models (LLMs).

Using Tavily can offer several benefits. Firstly, Tavily is optimized for LLMs, providing highly relevant, factual, and contextually appropriate search results. This optimization helps in reducing hallucinations and improving the overall decision-making capabilities of AI agents. Secondly, Tavily supports retrieval-augmented generation (RAG), which is crucial for ensuring the accuracy and relevance of information used by LLMs.

Moreover, Tavily’s seamless integration with popular AI frameworks like LangChain and LlamaIndex ensures that developers can easily incorporate it into their existing workflows. The efficiency and speed of Tavily’s search capabilities, combined with its comprehensive coverage by aggregating data from multiple sources, make it an invaluable tool for AI-driven applications. While DuckDuckGo remains a practical and cost-effective choice for many users, exploring Tavily’s advanced features can provide significant enhancements to the performance and accuracy of your AI models​.

The following code shows a sample search query presented to Tavily. The LLM would process these results to give it additional information to form a response.




In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults()
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'url': 'https://www.today.com/news/what-is-burning-man-flood-death-rcna103231',
  'content': '"\nAn update on BurningMan.org posted the night of Sept. 3 offered those still on site a "Wet Playa Survival Guide" with tips, as well as an update saying that conditions were "improving" and that roads in Black Rock City were expected to open on Sept. 4 for "Exodus."\n"Thank goodness this community knows how to take care of each other," the Instagram page for Burning Man Information Radio wrote on a post predicting more rain.\nNeal Katyal, Chris Rock and others detail their ‘escape’ from Black Rock City\nComedian Chris Rock and Diplo, producer and DJ,\xa0were among this year\'s Burning Man attendees. Why are people stranded at Burning Man?\nEach year participants in the Burning Man event gather for nine days in an "annual experiment in temporary community dedicated to Radical Self-expression and Radical Self-reliance," according to burningman.org.\nHeld in Nevada since 1991, the festival se

We now see that we can add it as a tool exactly like we used DuckDuckGo.

In [5]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

search_tool = TavilySearchResults()

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [search_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "Who is the oldest world leader?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'oldest world leader 2021'}`


[{'url': 'https://www.pewresearch.org/short-reads/2024/05/01/as-biden-and-trump-seek-reelection-who-are-the-oldest-and-youngest-current-world-leaders/', 'content': "Joe Biden, at 81, is the oldest American president, a distinction he's held since entering office at age 78. As Biden runs for reelection in 2024, he is the ninth oldest national leader in the world, according to a Pew Research Center analysis of sitting leaders in 187 United Nations member states. Former President Donald Trump, who is ..."}, {'url': 'https://en.wikipedia.org/wiki/List_of_oldest_living_state_leaders', 'content': 'The oldest living former state leader is Guillermo Rodríguez of Ecuador at the age of 100 years, 236 days. Leaders currently in office are in bold in green, with Paul Biya of Cameroon being the oldest currently serving head of state. Gallery. ... 11 Jan 2021: 12:'}, {'url':

{'input': 'Who is the oldest world leader?',
 'output': 'The oldest world leader in 2021 is Paul Biya of Cameroon.'}

## Using Agents with Retrival

We can also utilize retrieval for agents. We will use the RAG system that we previously used to create randomly generated biographical sketches for people working at five fictional companies. We use the same steps previously covered to load these documents into a LangChain retriever.

In [6]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain import OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.inmemory import InMemoryVectorStore
from langchain.schema import Document
import requests

urls = [
    "https://data.heatonresearch.com/data/t81-559/bios/DD.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/FT.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/GS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/NGS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/TI.txt"
]

def chunk_text(text, chunk_size, overlap):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

chunk_size = 900
overlap = 300

documents = []

for url in urls:
    print(f"Reading: {url}")
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    content = response.text
    chunks = chunk_text(content, chunk_size, overlap)
    for chunk in chunks:
        document = Document(page_content=chunk)
        documents.append(document)

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.vectorstores import Chroma

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

Reading: https://data.heatonresearch.com/data/t81-559/bios/DD.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/FT.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/GS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/NGS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/TI.txt


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

We test to see if the retriever can find text associated with one of the ficicious employees, Samantha Doyle.

In [7]:
retriever.invoke("who is Samantha Doyle")[0]

Document(page_content='the next generation of female tech leaders.\n\nSamantha Doyle is a seasoned Project Manager at Global Solutions, an innovative tech company known for pioneering smart city technologies. With over a decade of experience in the tech industry, Samantha has played a pivotal role in leading her team through successful launches of multiple high-profile sustainability projects. She holds a Masterâ\x80\x99s degree in Systems Engineering from MIT and has a passion for integrating eco-friendly practices into urban development. Outside of her professional life, Samantha is an avid rock climancer and enjoys mentoring young women interested in STEM careers, often volunteering her time at local high schools and community centers. Her dedication to both her career and community has made her a respected leader at Global Solutions and an influential figure in her field.\n\nSamantha Clarke is a seasoned Project Ma')

We now construct the tool, as you can see, we also give the agent instructions on when it should make use of this retriever.

In [8]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about people who work for several companies. For any questions about people you do not know, you must use this tool!",
)

We can now ask the agent a question that uses both search and the RAG documents. The following question:

> What are the job prospects in 2024 for Samantha Doyle's career? Return just a 2-sentence assessment of her career future.

This question requires the RAG data to determine Samantha's job. It then uses the search engine to find the current 2024 prospects of this job.

In [9]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

MODEL = 'gpt-4o-mini'

search_tool = search_tool = DuckDuckGoSearchRun()

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [search_tool, retriever_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What are the job prospects in 2024 for Samantha Doyle's career? Return just a 2-sentence assessment of her career future."})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'job prospects for Samantha Doyle in 2024'}`


men interested in STEM careers, aiming to inspire the next generation of female tech leaders.

Samantha Doyle is a seasoned Project Manager at Global Solutions, an innovative tech company known for pioneering smart city technologies. With over a decade of experience in the tech industry, Samantha has been instrumental in leading her team through several high-profile international projects, significantly enhancing urban infrastructure and sustainability in cities across Europe and Asia. A graduate of MIT with a degree in Systems Engineering, her expertise lies in integrating complex systems with emerging technologies to create seamless, efficient solutions. Outside of her professional life, Samantha is an avid rock climber and volunteers her time mentoring young women interested in STEM careers, aiming to inspire the next generation of female tech leaders.

{'input': "What are the job prospects in 2024 for Samantha Doyle's career? Return just a 2-sentence assessment of her career future.",
 'output': 'Samantha Doyle, a seasoned Project Manager at Global Solutions, is well-positioned for a successful career future in 2024. With over a decade of experience in leading high-profile international projects and expertise in integrating complex systems with emerging technologies, her innovative approach and leadership skills are likely to continue driving her success in the tech industry. Additionally, her dedication to mentoring young women in STEM careers showcases her commitment to inspiring the next generation of female tech leaders, further solidifying her impact and influence in the field.'}